# scMetabolism

In [ ]:
library(scMetabolism)
library(Seurat)
library(ggplot2)
library(rsvd)
library(pheatmap)
library(dplyr)

In [ ]:
setwd("/disk213/xieqq/JINHUA138.sc/scMetabolism")

In [ ]:
pbmc = readRDS("/disk213/xieqq/JINHUA138.sc/RDS/Epithelial.rds")
#level=c('Enterocytes','Colonocytes','BEST4 enterocytes','EECs','Tuft','Goblet','Progenitor','TA','Stem')
pbmc <- subset(pbmc, subset = CellType %in% c('Enterocytes','Colonocytes'))
pbmc@meta.data$SEGMENT.TIME <- factor(pbmc@meta.data$SEGMENT.TIME, 
                                      levels=c("duodenum-0","duodenum-60","duodenum-90","duodenum-180","duodenum-240",
                                               "jejunum-0","jejunum-60","jejunum-90","jejunum-180","jejunum-240",
                                               "ileum-0","ileum-60","ileum-90","ileum-180","ileum-240",
                                               "cecum-0","cecum-60","cecum-90","cecum-180","cecum-240",
                                               "colon-0","colon-60","colon-90","colon-180","colon-240"),
                                      labels=c("01_duodenum-0","02_duodenum-60","03_duodenum-90","04_duodenum-180","05_duodenum-240",
                                               "06_jejunum-0","07_jejunum-60","08_jejunum-90","09_jejunum-180","10_jejunum-240",
                                               "11_ileum-0","12_ileum-60","13_ileum-90","14_ileum-180","15_ileum-240",
                                               "16_cecum-0","17_cecum-60","18_cecum-90","19_cecum-180","20_cecum-240",
                                               "21_colon-0","22_colon-60","23_colon-90","24_colon-180","25_colon-240"))

In [ ]:
countexp.Seurat <- sc.metabolism.Seurat(obj = pbmc,
                                        method = "AUCell",  #ssGSEA AUCell VISION GSVA
                                        imputation = F,
                                        ncores = 2,
                                        metabolism.type = "KEGG")

可视化

In [ ]:
####heatmap
metabolism <- countexp.Seurat
df = data.frame(t(metabolism@assays[["METABOLISM"]][["score"]]))
# rownames(df)
# names(metabolism$`SEGMENT.TIME`) 
rownames(df) <- gsub(".", "-", rownames(df), fixed = TRUE)
df = df[names(metabolism$`SEGMENT.TIME`),]
df$orig.ident <- metabolism$`SEGMENT.TIME`
write.csv(df, "scMetabolism.csv")

avg_df =aggregate(df[,1:ncol(df)-1],list(df$orig.ident),mean)
rownames(avg_df) = avg_df$Group.1
avg_df=avg_df[,-1]
avg_df <- as.data.frame(t(avg_df))

data <- avg_df[,c("duodenum-0","duodenum-60","duodenum-90","duodenum-180","duodenum-240",
                  "jejunum-0","jejunum-60","jejunum-90","jejunum-180","jejunum-240",
                  "ileum-0","ileum-60","ileum-90","ileum-180","ileum-240",
                  "cecum-0","cecum-60","cecum-90","cecum-180","cecum-240",
                  "colon-0","colon-60","colon-90","colon-180","colon-240")]
write.csv(data, "scMetabolism.matrix.csv")

kegg_inf <- read.csv("/disk213/xieqq/Software/KEGG/KEGG_map.csv", row.names=1, check.names=F)
metabolism_inf <- read.csv("scMetabolism.inf.csv", check.names=F)
metabolism_inf <- inner_join(metabolism_inf,kegg_inf[,c(1,4)],by=c("new_name"="KEGG"))

data <- subset(data,rownames(data) %in% metabolism_inf$old_name)
old_names <- metabolism_inf$old_name
new_names <- metabolism_inf$new_name
match_pos <- match(old_names, rownames(data))
rownames(data)[match_pos] <- new_names[match_pos]

annotation_row <- data.frame(Category=metabolism_inf$second_category)
rownames(annotation_row) <- metabolism_inf$new_name

annotation_col <- data.frame(Segment=sapply(strsplit(colnames(data), "-"), function(x) x[1]),
                             Time=sapply(strsplit(colnames(data), "-"), function(x) x[2]))
annotation_col$Time <- paste0(annotation_col$Time,"d")
rownames(annotation_col) <- colnames(data)

ann_colors=list(Segment=c('duodenum'='#E76F51','jejunum'='#F4A261','ileum'='#E9C46A','cecum'='#2A9D8F','colon'='#264653'),
                Time=c('0d'='#FF595E','60d'='#FFCA3A','90d'='#8AC926','180d'='#1982C4','240d'='#6A4C93'))

pheatmap(data, scale="row", 
         annotation_colors=ann_colors, 
         annotation_col=annotation_col,
         annotation_row=annotation_row,
         show_rownames=T, show_colnames=T, cluster_rows=F, cluster_cols=F,
         color=colorRampPalette(c("navy",'white',"firebrick3"))(50),
         filename="pheatmap.pdf", width=12, height=12)

In [ ]:
####Dotplot
kegg_inf <- read.csv("/disk213/xieqq/Software/KEGG/KEGG_map.csv", row.names=1, check.names=F)
input.pathway <- data.frame(KEGG=rownames(countexp.Seurat@assays[["METABOLISM"]][["score"]]))
input.pathway <- left_join(input.pathway,kegg_inf[,c(1,4)],by="KEGG")
input.pathway$inf <- paste0(input.pathway$second_category,"_",input.pathway$KEGG)
rownames(countexp.Seurat@assays[["METABOLISM"]][["score"]]) <- input.pathway$inf

p <- DotPlot.metabolism(obj = countexp.Seurat, 
                        pathway = input.pathway$inf, 
                        phenotype = "SEGMENT.TIME", 
                        norm = "y")
pdf(file="DotPlot.pdf", width=12, height=18)
p
dev.off()

P <- p +
     scale_color_gradient(limits=c(0,1), high="firebrick3", low="white")+
     labs(x="", y="", title="")+
     theme_bw()+
     theme(axis.text.x=element_text(angle=90,color="black",size=12),
           axis.text.y=element_text(color="black",size=12),
           axis.title.x=element_text(color="black",size=16),
           axis.title.y=element_text(color="black",size=16),
           legend.text=element_text(color="black",size=12),
           legend.title=element_text(color="black",size=16),
           panel.grid.major=element_blank(),
           panel.grid.minor=element_blank())
pdf(file="DotPlot1.pdf", width=15, height=18)
P
dev.off()

In [ ]:
BX <- BoxPlot.metabolism(obj = countexp.Seurat, pathway = input.pathway$inf[c(34:46)], phenotype = "SEGMENT.TIME", ncol = 2)
BX <- BX+
      labs(x="", y="", title="")+
      theme_bw()+
      theme(axis.text.x=element_blank(),
            axis.text.y=element_text(color="black",size=12),
            axis.title.x=element_text(color="black",size=16),
            axis.title.y=element_text(color="black",size=16),
            legend.text=element_text(color="black",size=12),
            legend.title=element_text(color="black",size=16),
            panel.grid.major=element_blank(),
            panel.grid.minor=element_blank())
# ggsave("BoxPlot_C.pdf", plot=BX, width=15, height=14)
# ggsave("BoxPlot_E.pdf", plot=BX, width=15, height=4)
# ggsave("BoxPlot_L.pdf", plot=BX, width=15, height=16)
ggsave("BoxPlot_A.pdf", plot=BX, width=15, height=14)

In [ ]:
for (i in c("Carbohydrate metabolism","Energy metabolism","Lipid metabolism","Amino acid metabolism")){
      BX <- BoxPlot.metabolism(obj = countexp.Seurat, pathway = input.pathway$inf[which(input.pathway$second_category==i)], phenotype = "SEGMENT.TIME", ncol = 2)
      BX <- BX + labs(x="", y="", title="")+
      theme_bw()+
      theme(axis.text.x=element_blank(),
            axis.text.y=element_text(color="black",size=12),
            axis.title.x=element_text(color="black",size=16),
            axis.title.y=element_text(color="black",size=16),
            legend.text=element_text(color="black",size=12),
            legend.title=element_text(color="black",size=16),
            panel.grid.major=element_blank(),
            panel.grid.minor=element_blank())
      ggsave(paste0("BoxPlot_",i,".pdf"), plot=BX, width=15, height=14)
}